<img src="PEST++V3_cover.jpeg" style="float: left">

<img src="flopylogo.png" style="float: right">

<img src="AW&H2015.png" style="float: center">

# Looking at Parameter Sensitivity

We have already discussed the Jacobian matrix in a few places. It is calculated by perturbing the parameter (usually 1%) and tracking what happens to each observation.  In a general form the sensitivity equation looks like eq. 9.7 Anderson et al. 2015:

<img src="Sensitivity_eq.png" style="float: center">

This is key for derivative-based parameter estimation because, as we've seen, this allows us to efficiently compute upgraded parameters to try during the lambda search.  But the Jacobian matrix can give us insight about the model in and of itself. 

Let's take a look at it more closely and see what we can learn from it and how to handle such information as the number of parameters rises.

In [1]:
%matplotlib inline
import os
import sys
sys.path.append('..')
import shutil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pyemu


Bring in the model...

In [2]:
import freyberg_setup
freyberg_setup.setup_pest_kr()
working_dir = freyberg_setup.WORKING_DIR_KR
pst_name = freyberg_setup.PST_NAME_KR

['.DS_Store', 'botm.ref', 'extract_zone_array.py', 'forecasts_true.csv', 'freyberg.bas', 'freyberg.dbf', 'freyberg.dis', 'freyberg.hds', 'freyberg.heads', 'freyberg.heads_potobs.ins', 'freyberg.hyd', 'freyberg.list', 'freyberg.locations', 'freyberg.mpbas', 'freyberg.mpenpt', 'freyberg.mplist', 'freyberg.mpnam', 'freyberg.mppthln', 'freyberg.mpsim', 'freyberg.oc', 'freyberg.pcg', 'freyberg.rivflux', 'freyberg.shp', 'freyberg.shx', 'freyberg.travel', 'freyberg.truth.lpf', 'freyberg.truth.nam', 'freyberg.truth.rch', 'freyberg.truth.riv', 'freyberg.truth.wel', 'hk.truth.ref', 'hk.zones', 'ibound.ref', 'inschek', 'inschek.exe', 'kzone.ref', 'mf2005', 'mf2005.exe', 'mfnwt', 'mp6', 'mp6.exe', 'mpath.in', 'obs_loc.csv', 'pest++.exe', 'pestchek', 'pestchek.exe', 'pestpp', 'potobs_group.csv', 'Process_output.py', 'run_true_model.py', 'strt.ref', 'sweep', 'sweep.exe', 'tempchek', 'tempchek.exe', 'Weights_and_best_PHI.xlsx']

changing model workspace...
   freyberg_kr
FloPy is using the following 

## First read in the PST file and find what are the starting values for K and R

In [3]:
inpst = pyemu.Pst(os.path.join(working_dir,pst_name))
inpst.parameter_data

,parnme,partrans,parchglim,parval1,parlbnd,parubnd,pargp,scale,offset,dercom
parnme,,,,,,,,,,
rch_0,rch_0,fixed,factor,1.0,0.5,2.0,rch,1.0,0.0,1
rch_1,rch_1,fixed,factor,1.0,0.5,2.0,rch,1.0,0.0,1
hk,hk,log,factor,5.0,0.5,50.0,hk,1.0,0.0,1


### Now let's tell PEST++ to calculate the Jacobian matrix by changing NOPTMAX from 0 to -1 
(recall NOPTMAX=-1 calculates the Jacobian but not all the statistics, which is NOPTMAX=-2)

In [7]:
inpst = pyemu.Pst(os.path.join(working_dir,pst_name))
inpst.control_data.noptmax =  -1
inpst.write(os.path.join(working_dir,pst_name))

### Now let's calculate the sensitivity by PEST++
(recall a Jacobian matrix takes a minimum of NPAR + 1, which is 4 runs for this case)

In [8]:
 pyemu.helpers.run("pestpp {0}".format(pst_name.replace(".pst",".final.pst")))

In [ ]:
# observation locations
obslox = pd.read_csv('freyberg.hyd', delim_whitespace=True, usecols = [4,5,6], 
                     index_col=2, skiprows = 1, header=None, names=['X','Y','obsname'])
# parameter locations
parlox = pd.read_csv('points1.dat.tpl', delim_whitespace=True, usecols=[0,1,2],
                    index_col=0, skiprows=1, header=None, names=['parname','X','Y'])

In [ ]:
def plot_Jacobian(jac, figsize, cmap='viridis',logtrans=True):
    f = plt.figure(figsize=figsize)
    ax = plt.axes([0, 0.05, 0.9, 0.9 ]) #left, bottom, width, height
    if logtrans:
        jcdata=np.log(np.abs(jac.df()))
    else:
        jcdata=jac.df()
    im = ax.imshow(jcdata, interpolation='nearest', cmap=cmap, aspect='auto')
    plt.xticks(range(len(jac.col_names)), jac.col_names, rotation=90)
    plt.yticks(range(len(jac.row_names)), jac.row_names)
    
    ax.grid(False)
    cax = plt.axes([0.95, 0.05, 0.05,0.9 ])
    plt.colorbar(mappable=im, cax=cax)

In [ ]:
def spatial_plot_sens(jac,cobs, obslox, parlox, figsize=(4,7)):
    sens = jac.df().loc[cobs]
    sens.drop('rch1',inplace=1)
    fig=plt.figure(figsize=figsize)
    plt.plot(parlox.X,parlox.Y,'kd',markersize=.8)
    scalefactor=5
    if 'flux' not in cobs:
        coblox = obslox.loc[cobs]
        plt.plot(coblox.X,coblox.Y,'kx', markersize=10)
        scalefactor=1000    
    plt.scatter(parlox.X,parlox.Y, s=np.abs(sens.values)*scalefactor, c=sens.values, cmap='viridis')
    plt.axis('equal')
    plt.colorbar()
    plt.title('Sensitivity for {0}'.format(cobs))
    plt.xlim(0,5000)
    plt.ylim(0,10000)
    plt.axis('off')
    return fig

### Look at the Jacobian matrix---gradients of parameters wrt. observations

For each parameter-observation combination, we can see how much the observation value changes due to a small change in the parameter. If $y$ are the observations and $x$ are the parameters, the equation for the $i^th$ observation with respect to the $j^th$ parameter is:  
## $\frac{\partial y_i}{\partial x_j}$
This can be approximated by finite differences as :  
## $\frac{\partial y_i}{\partial x_j}~\frac{y\left(x+\Delta x \right)-y\left(x\right)}{\Delta x}$

### First we can read in a couple Jacobian matrices -- one from our simple model, and one from a more complex one

In [ ]:
jac_simple = pyemu.Jco.from_binary(os.path.join('..','..','models','Freyberg','Freyberg_K_and_R','freyberg.jcb'))
jac_complex = pyemu.Jco.from_binary(os.path.join('..','..','models','Freyberg','Freyberg_pilotpoints','freyberg_pp_reg_phimlim26.jcb'))

### These are now matrices. How big are they?

In [ ]:
print ('simple  --> {0} rows x {1} columns'.format(*jac_simple.shape))
print ('complex --> {0} rows x {1} columns'.format(*jac_complex.shape))


In [ ]:
plot_Jacobian(jac_simple, figsize=(7,4))

In [ ]:
# Let's drop all the forecasts and regularization information
jac_simple.drop([x for x in jac_simple.df().index if x.startswith('fr')], axis=0)
jac_simple.drop([x for x in jac_simple.df().index if 'fore' in x], axis=0)
jac_simple.drop('travel_time', axis=0)

In [ ]:
plot_Jacobian(jac_simple, figsize=(7,4))

In [ ]:
plot_Jacobian(jac_complex, figsize=(7,4))

In [ ]:
# Let's drop all the forecasts and regularization information
jac_complex.drop([x for x in jac_complex.df().index if x.startswith('i')], axis=0)
jac_complex.drop([x for x in jac_complex.df().index if x.startswith('fr')], axis=0)
jac_complex.drop([x for x in jac_complex.df().index if 'fore' in x], axis=0)
jac_complex.drop('travel_time', axis=0)

In [ ]:
plot_Jacobian(jac_complex, figsize=(7,4))

# Can be more informative to look at sensitivity spatially

In [ ]:
print(jac_complex.row_names)

In [ ]:
spatial_plot_sens(jac_complex,'cr04c9', obslox,parlox);

In [ ]:
with PdfPages('allsens.pdf') as ofp:
    for cob in jac_complex.row_names:
        cf = spatial_plot_sens(jac_complex, cob, obslox,parlox)
        ofp.savefig()
        plt.close('all')

# How about Composite Scaled Sensitivities
In the traditional, overdetermined regression world, CSS was a popular metric. CSS is Composite Scaled Sensitivitity.

In Hill and Tiedeman (2007) this is calculated as: 
## ${css_{j}=\sqrt{\left(\sum_{i-1}^{ND}\left(\frac{\partial y'_{i}}{\partial b_{j}}\right)\left|b_{j}\right|\sqrt{w_{ii}}\right)/ND}}$

In PEST, Doherty calculates it slightly differently in that scaling by the parameter values happens automatically when the parameter is subjected to a log-transform. This is due to a correction that must be made in calculating the Jacobian matrix and follows from the chain rule of derivatives.


In [ ]:
la = pyemu.LinearAnalysis(jco=os.path.join('..','..','models','Freyberg','Freyberg_pilotpoints','freyberg_pp_reg_phimlim26.jcb'))

In [ ]:
plt.figure(figsize=(8,4))
ax = la.get_par_css_dataframe()['pest_css'].sort_values(ascending=False).plot(kind='bar')
ax.set_yscale('log')

### Now let's consider correlation and posterior covariance

In [ ]:
sc = pyemu.Schur(os.path.join('..','..','models','Freyberg','Freyberg_pilotpoints','freyberg_pp_reg_phimlim26.jcb'))
covar = pyemu.Cov(sc.xtqx.x, names=sc.xtqx.row_names)
covar.df().head()

In [ ]:
R = covar.to_pearson()
plt.imshow(R.df(), interpolation='nearest', cmap='viridis')
plt.colorbar()

In [ ]:
cpar = 'hkpp10'
R.df().loc[cpar][np.abs(R.df().loc[cpar])>.5]

In [ ]:
R_plot = R.df().as_matrix()
R_plot[np.abs(R_plot)>.9] = np.nan
plt.imshow(R_plot, interpolation='nearest', cmap='viridis')
plt.colorbar()